# Report (Tým 12)

Původně jsme kód napsali podle pseudokódu ze zadání. Později jsme ale přišli na to, že pseudokód v zadání není úplný (možná úmyslně?) a vlastně ani nesplňuje zadání samotný.

Kód jsme upravili, aby pracoval s relativní přechodovou matici místo absolutní.

## Funkce `prolom_substitute`

Pseudokód vrací klíč z poslední iterace algoritmu, což ale nedává dobré výsledky, protože se ve spoustě případech nejedná o nejlepší nalezený klíč. Proto jsme kód upravili, aby si pamatoval a následně vracel nejlepší nalezený klíč za celý běh.

Pro pravděpodobnost přijetí kandidátního klíče s nižší věrohodností se nám osvědčila nižší hodnota, než je v pseudokódu. Konkrétně používáme pravděpodobnost 0.001.

## Optimalizace

Původně byla naše kryptoanalýza velmi pomalá a museli jsme tedy v kódu najít místa, která lze optimalizovat. Našli jsme dvě taková místa.

### Funkce `transition_matrix`

V naší původní funkci (vložená níže bez docstringu) trvalo dlouho postavení absolutní přechodové matice. To protože přístup a úprava jednotlivým prvkům v Pandas Data Framu je (pro naše účely) velmi pomalá.

```python
def transition_matrix(bigrams: list[str]) -> pd.DataFrame:
    n = len(alphabet)

    alphabet_list = list(alphabet)
    TM = pd.DataFrame(
        np.zeros((n, n), dtype=int),
        index=alphabet_list,
        columns=alphabet_list
    )

    # Build the absolute transition matrix
    for bigram in bigrams:
        c1 = bigram[0]
        c2 = bigram[1]
        if c1 in alphabet and c2 in alphabet:
            i = alphabet.index(c1)
            j = alphabet.index(c2)
            TM.iat[i, j] += 1

    # Replace zeros with ones
    TM.replace(0, 1, inplace=True)

    # Convert to relative transition matrix
    TM = TM.div(TM.sum().sum())

    return TM
```

To jsme vyřešili tak, že nejdříve absolutní přechodovou matici vytvoříme jako NumPy Array a až poté ji převedeme na Pandas Data Frame.

```python
def transition_matrix(bigrams: list[str]) -> pd.DataFrame:
    n = len(alphabet)
    arr = np.zeros((n, n), dtype=int)

    # Build the absolute transition matrix
    for bigram in bigrams:
        c1 = bigram[0]
        c2 = bigram[1]
        if c1 in alphabet and c2 in alphabet:
            i = alphabet.index(c1)
            j = alphabet.index(c2)
            arr[i, j] += 1

    alphabet_list = list(alphabet)
    TM = pd.DataFrame(
        arr,
        index=alphabet_list,
        columns=alphabet_list
    )

    # Replace zeros with ones
    TM.replace(0, 1, inplace=True)

    # Convert to relative transition matrix
    TM = TM.div(TM.sum().sum())

    return TM
```

Rychlý benchmark obou funkcí ukazuje, že optimalizovaná verze je přibližně 25 krát rychlejší!

### Funkce `plausibility`

V případě této funkce se také přistupuje k jednotlivým prvkům Pandas Data Framu, což je zde opět slabé místo.

```python
def plausibility(text: str, TM_ref: pd.DataFrame) -> float:
    bigrams_obs = get_bigrams(text)
    TM_obs = transition_matrix(bigrams_obs)

    likelihood = 0.0
    for i in range(len(alphabet)):
        for j in range(len(alphabet)):
            likelihood += math.log(TM_ref.iat[i, j]) * TM_obs.iat[i, j]

    return likelihood
```

Pro optimalizaci jsme zde opět použili NumPy.

```python
def plausibility(text: str, TM_ref: pd.DataFrame) -> float:
    bigrams_obs = get_bigrams(text)
    TM_obs = transition_matrix(bigrams_obs)

    TM_ref = np.asarray(TM_ref)
    TM_obs = np.asarray(TM_obs)
    likelihood = np.sum(np.log(TM_ref) * TM_obs)

    return likelihood
```

V benchmarku jsme pro obě verze použili již optimalizovanou `transition_matrix` funkci. Zde tedy rychlý benchmark ukazuje, že optimalizovaná verze je přibližně 15 krát rychlejší!

### Závěr optimalizace

Protože funkce `prolom_substitute` volá v každé iteraci funkci `plausibility`, která následně volá funkci `transition_matrix`, tak tyto dvě optimalizace značně zrychlily průběh kryptoanalýzy. Konkrétně volání funkce `prolom_substitute` s textem z `text_1000_sample_1_ciphertext.txt` a 20 000 iterací se snížilo z 12m 38s na 24.8s. To je přibližně 30 násobné zrychlení!

## Dosažené výsledky

Naše knihovna úspěšně umí zašifrovat a dešifrovat text pomocí substituční šifry. Také umí prolomit text šifrovaný substituční šifrou pomocí Metropolis-Hastings (M-H) algoritmu. Demonstrace naší knihovny včetně analýzy výsledků prolomení testovacích šifrovaných textů je v souboru `demo.ipynb`.

## Členové týmu 12

- Jiří Hošek
- Martin Nebehay
- Jiří Mrkvica
- Samuel Všelko